# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Iterative-Methods-for-Solving-Linear-Equations" data-toc-modified-id="Iterative-Methods-for-Solving-Linear-Equations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Iterative Methods for Solving Linear Equations</a></div><div class="lev2 toc-item"><a href="#Introduction" data-toc-modified-id="Introduction-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></div><div class="lev2 toc-item"><a href="#PageRank-problem" data-toc-modified-id="PageRank-problem-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>PageRank problem</a></div><div class="lev2 toc-item"><a href="#Jacobi-method" data-toc-modified-id="Jacobi-method-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Jacobi method</a></div><div class="lev2 toc-item"><a href="#Gauss-Seidel-method" data-toc-modified-id="Gauss-Seidel-method-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Gauss-Seidel method</a></div><div class="lev2 toc-item"><a href="#Successive-over-relaxation-(SOR)" data-toc-modified-id="Successive-over-relaxation-(SOR)-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Successive over-relaxation (SOR)</a></div><div class="lev2 toc-item"><a href="#Conjugate-gradient-method" data-toc-modified-id="Conjugate-gradient-method-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Conjugate gradient method</a></div><div class="lev2 toc-item"><a href="#MatrixDepot.jl" data-toc-modified-id="MatrixDepot.jl-17"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>MatrixDepot.jl</a></div><div class="lev2 toc-item"><a href="#Numerical-examples" data-toc-modified-id="Numerical-examples-18"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Numerical examples</a></div><div class="lev3 toc-item"><a href="#Generate-test-matrix" data-toc-modified-id="Generate-test-matrix-181"><span class="toc-item-num">1.8.1&nbsp;&nbsp;</span>Generate test matrix</a></div><div class="lev2 toc-item"><a href="#Matrix-vector-muliplication" data-toc-modified-id="Matrix-vector-muliplication-19"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Matrix-vector muliplication</a></div><div class="lev3 toc-item"><a href="#Dense-solve-via-Cholesky" data-toc-modified-id="Dense-solve-via-Cholesky-191"><span class="toc-item-num">1.9.1&nbsp;&nbsp;</span>Dense solve via Cholesky</a></div><div class="lev3 toc-item"><a href="#Jacobi-solver" data-toc-modified-id="Jacobi-solver-192"><span class="toc-item-num">1.9.2&nbsp;&nbsp;</span>Jacobi solver</a></div><div class="lev2 toc-item"><a href="#Gauss-Seidal" data-toc-modified-id="Gauss-Seidal-110"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Gauss-Seidal</a></div><div class="lev3 toc-item"><a href="#SOR" data-toc-modified-id="SOR-1101"><span class="toc-item-num">1.10.1&nbsp;&nbsp;</span>SOR</a></div><div class="lev3 toc-item"><a href="#Conjugate-Gradient" data-toc-modified-id="Conjugate-Gradient-1102"><span class="toc-item-num">1.10.2&nbsp;&nbsp;</span>Conjugate Gradient</a></div>

In [1]:
versioninfo()

Julia Version 1.1.0
Commit 80516ca202 (2019-01-21 21:24 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = code


# Iterative Methods for Solving Linear Equations

## Introduction

So far we have considered direct methods for solving linear equations.    

* **Direct methods** (flops fixed _a priori_) vs **iterative methods**:
    - Direct method (GE/LU, Cholesky, QR, SVD): good for dense, small to moderate sized $\mathbf{A}$  
    - Iterative methods (Jacobi, Gauss-Seidal, SOR, conjugate-gradient, GMRES): good for large, sparse, structured linear system, parallel computing, warm start


## PageRank problem

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/PageRanks-Example.svg/400px-PageRanks-Example.svg.png" width="300" align="center"/>

* $\mathbf{A}  \in \{0,1\}^{n \times n}$ the connectivity matrix of webpages with entries
$$
\begin{eqnarray*}
	a_{ij} = \begin{cases}
	1 &  \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}. 
\end{eqnarray*}
$$
$n \approx 10^9$ in May 2017.

* $r_i = \sum_j a_{ij}$ is the *out-degree* of page $i$. 

* [Larry Page](https://en.wikipedia.org/wiki/PageRank) imagines a random surfer wandering on internet according to following rules:
    - From a page $i$ with $r_i>0$
        * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
        * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page 
    - From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page
    
The process defines a Markov chain on the space of $n$ pages. Stationary distribution of this Markov chain gives the ranks (probabilities) of each page.

* Stationary distribution is the top left eigenvector of the transition matrix $\mathbf{P}$ corresponding to eigenvalue 1. Equivalently it can be cast as a linear equation.
$$
    (\mathbf{I} - \mathbf{P}^T) \mathbf{x} = \mathbf{0}.
$$

* Gene Golub: Largest matrix computation in world. 

* GE/LU will take $2 \times (10^9)^3/3/10^{12} \approx 6.66 \times 10^{14}$ seconds $\approx 20$ million years on a tera-flop supercomputer!

* Iterative methods come to the rescue.

## Jacobi method

<img src="https://www.usna.edu/Users/math/meh/Jacobi.jpeg" width="150" align="center"/>

Solve $\mathbf{A} \mathbf{x} = \mathbf{b}$.

* Jacobi iteration: 
$$
x_i^{(t+1)} = \frac{b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)}}{a_{ii}}.
$$

* With splitting: $\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$, Jacobi iteration can be written as
$$
    \mathbf{D} \mathbf{x}^{(t+1)} = - (\mathbf{L} + \mathbf{U}) \mathbf{x}^{(t)} + \mathbf{b},
$$
i.e., 
$$
	\mathbf{x}^{(t+1)} = - \mathbf{D}^{-1} (\mathbf{L} + \mathbf{U}) \mathbf{x}^{(t)} + \mathbf{D}^{-1} \mathbf{b} = - \mathbf{D}^{-1} \mathbf{A} \mathbf{x}^{(t)} + \mathbf{x}^{(t)} + \mathbf{D}^{-1} \mathbf{b}.
$$

* One round costs $2n^2$ flops with an **unstructured** $\mathbf{A}$. Gain over GE/LU if converges in $o(n)$ iterations. Saving is huge for **sparse** or **structured** $\mathbf{A}$. By structured, we mean the matrix-vector multiplication $\mathbf{A} \mathbf{v}$ is fast ($O(n)$ or $O(n \log n)$).

## Gauss-Seidel method

<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/Carl_Friedrich_Gauss.jpg" width="150" align="center"/>

<img src="http://www.scientificlib.com/en/Physics/Biographies/images/Thumbs/ThLudwigVonSeidel01.jpg" width="150" align="center"/>

* Gauss-Seidel (GS) iteration:
$$
x_i^{(t+1)} = \frac{b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t+1)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)}}{a_{ii}}.
$$

* With splitting, $(\mathbf{D} + \mathbf{L}) \mathbf{x}^{(t+1)} = - \mathbf{U} \mathbf{x}^{(t)} + \mathbf{b}$, i.e., 
$$
\mathbf{x}^{(t+1)} = - (\mathbf{D} + \mathbf{L})^{-1} \mathbf{U} \mathbf{x}^{(t)} + (\mathbf{D} + \mathbf{L})^{-1} \mathbf{b}.
$$

* GS converges for any $\mathbf{x}^{(0)}$ for symmetric and pd $\mathbf{A}$.

* Convergence rate of Gauss-Seidel is the spectral radius of the $(\mathbf{D} + \mathbf{L})^{-1} \mathbf{U}$.

## Successive over-relaxation (SOR)

* SOR: 
$$
x_i^{(t+1)} = \omega \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t+1)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)} \right) / a_{ii} + (1-\omega) x_i^{(t)},
$$
i.e., 
$$
\mathbf{x}^{(t+1)} = (\mathbf{D} + \omega \mathbf{L})^{-1} [(1-\omega) \mathbf{D} - \omega \mathbf{U}] \mathbf{x}^{(t)} + (\mathbf{D} + \omega \mathbf{L})^{-1} (\mathbf{D} + \mathbf{L})^{-1} \omega \mathbf{b}.
$$

* Need to pick $\omega \in [0,1]$ beforehand, with the goal of improving convergence rate.

## Conjugate gradient method

* **Conjugate gradient and its variants are the top-notch iterative methods for solving huge, structured linear systems.**

* A UCLA invention! Hestenes.

* Solving $\mathbf{A} \mathbf{x} = \mathbf{b}$ is equivalent to minimizing the quadratic function $\frac{1}{2} \mathbf{x}^T \mathbf{A} \mathbf{x} - \mathbf{b}^T \mathbf{x}$. 

[Kershaw's results](http://www.sciencedirect.com/science/article/pii/0021999178900980?via%3Dihub) for a fusion problem.

| Method                                 | Number of Iterations |
|----------------------------------------|----------------------|
| Gauss Seidel                           | 208,000              |
| Block SOR methods                      | 765                  |
| Incomplete Cholesky conjugate gradient | 25                   |

## MatrixDepot.jl

MatrixDepot.jl is an extensive collection of test matrices in Julia. After installation, we can check available test matrices by

In [2]:
using MatrixDepot

mdinfo()

include group.jl for user defined matrix generators
verify download of index files...
used remote site is https://sparse.tamu.edu/?per_page=All
populating internal database...


### Currently loaded Matrices

| builtin(#)  |             |              |             |               |
|:----------- |:----------- |:------------ |:----------- |:------------- |
| 1 baart     | 13 fiedler  | 25 invhilb   | 37 parter   | 49 shaw       |
| 2 binomial  | 14 forsythe | 26 invol     | 38 pascal   | 50 smallworld |
| 3 blur      | 15 foxgood  | 27 kahan     | 39 pei      | 51 spikes     |
| 4 cauchy    | 16 frank    | 28 kms       | 40 phillips | 52 toeplitz   |
| 5 chebspec  | 17 gilbert  | 29 lehmer    | 41 poisson  | 53 tridiag    |
| 6 chow      | 18 golub    | 30 lotkin    | 42 prolate  | 54 triw       |
| 7 circul    | 19 gravity  | 31 magic     | 43 randcorr | 55 ursell     |
| 8 clement   | 20 grcar    | 32 minij     | 44 rando    | 56 vand       |
| 9 companion | 21 hadamard | 33 moler     | 45 randsvd  | 57 wathen     |
| 10 deriv2   | 22 hankel   | 34 neumann   | 46 rohess   | 58 wilkinson  |
| 11 dingdong | 23 heat     | 35 oscillate | 47 rosser   | 59 wing       |
| 12 erdrey   | 24 hilb     | 36 parallax  | 48 sampling |               |

| user(#) |
|:------- |

| Groups  |       |       |         |        |         |           |     |     |     |     |     |
|:------- |:----- |:----- |:------- |:------ |:------- |:--------- |:--- |:--- |:--- |:--- |:--- |
| all     | local | eigen | illcond | posdef | regprob | symmetric |     |     |     |     |     |
| builtin | user  | graph | inverse | random | sparse  |           |     |     |     |     |     |

| Suite Sparse | of   |
|:------------ |:---- |
| 0            | 2833 |

| MatrixMarket | of  |
|:------------ |:--- |
| 0            | 498 |


In [3]:
# List matrices that are positive definite and sparse:
mdlist(:symmetric & :posdef & :sparse)

2-element Array{String,1}:
 "poisson"
 "wathen" 

In [4]:
# Get help on Poisson matrix
mdinfo("poisson")

# Poisson Matrix (poisson)

A block tridiagonal matrix from Poisson’s equation.      This matrix is sparse, symmetric positive definite and      has known eigenvalues. The result is of type `SparseMatrixCSC`.

*Input options:*

  * [type,] dim: the dimension of the matirx is `dim^2`.

*Groups:* ["inverse", "symmetric", "posdef", "eigen", "sparse"]

*References:*

**G. H. Golub and C. F. Van Loan**, Matrix Computations,           second edition, Johns Hopkins University Press, Baltimore,           Maryland, 1989 (Section 4.5.4).


In [5]:
# Generate a Poisson matrix of dimension n=10
A = matrixdepot("poisson", 10)

100×100 SparseArrays.SparseMatrixCSC{Float64,Int64} with 460 stored entries:
  [1  ,   1]  =  4.0
  [2  ,   1]  =  -1.0
  [11 ,   1]  =  -1.0
  [1  ,   2]  =  -1.0
  [2  ,   2]  =  4.0
  [3  ,   2]  =  -1.0
  [12 ,   2]  =  -1.0
  [2  ,   3]  =  -1.0
  [3  ,   3]  =  4.0
  [4  ,   3]  =  -1.0
  [13 ,   3]  =  -1.0
  [3  ,   4]  =  -1.0
  ⋮
  [98 ,  97]  =  -1.0
  [88 ,  98]  =  -1.0
  [97 ,  98]  =  -1.0
  [98 ,  98]  =  4.0
  [99 ,  98]  =  -1.0
  [89 ,  99]  =  -1.0
  [98 ,  99]  =  -1.0
  [99 ,  99]  =  4.0
  [100,  99]  =  -1.0
  [90 , 100]  =  -1.0
  [99 , 100]  =  -1.0
  [100, 100]  =  4.0

In [6]:
using UnicodePlots
spy(A)

                    Sparsity Pattern
       ┌──────────────────────────────────────────┐    
     1 │⠻⣦⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
       │⠀⠈⠻⣦⡀⠀⠉⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
       │⠒⢄⠀⠈⠱⣦⡀⠀⠑⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠣⢄⠀⠈⠻⣦⡀⠀⠑⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠱⣀⠀⠈⠱⣦⡀⠀⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠑⡄⠀⠈⠻⣦⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠈⠑⢄⠀⠈⠱⣦⡀⠀⠉⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠒⢄⠀⠈⠻⣦⡀⠀⠑⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠣⢄⠀⠈⠛⣤⡀⠀⠑⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⣀⠀⠈⠻⣦⡀⠀⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⡄⠀⠈⠛⣤⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠑⢄⠀⠈⠻⣦⡀⠀⠉⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠒⢄⠀⠈⠛⣤⡀⠀⠑⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠣⢄⠀⠈⠻⣦⡀⠀⠑⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⣀⠀⠈⠻⢆⡀⠀⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⡄⠀⠈⠻⣦⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀

In [7]:
# Get help on Wathen matrix
mdinfo("wathen")

# Wathen Matrix (wathen)

Wathen Matrix is a sparse, symmetric positive, random matrix arose from the finite element method. The generated matrix is the consistent mass matrix for a regular nx-by-ny grid of 8-nodes.

*Input options:*

  * [type,] nx, ny: the dimension of the matrix is equal to   `3 * nx * ny + 2 * nx * ny + 1`.
  * [type,] n: `nx = ny = n`.

*Groups:* ["symmetric", "posdef", "eigen", "random", "sparse"]

*References:*

**A. J. Wathen**, Realistic eigenvalue bounds for     the Galerkin mass matrix, IMA J. Numer. Anal., 7 (1987),     pp. 449-457.


In [8]:
# Generate a Wathen matrix of dimension n=5
A = matrixdepot("wathen", 5)

96×96 SparseArrays.SparseMatrixCSC{Float64,Int64} with 1256 stored entries:
  [1 ,  1]  =  12.1111
  [2 ,  1]  =  -12.1111
  [3 ,  1]  =  4.03703
  [12,  1]  =  -12.1111
  [13,  1]  =  -16.1481
  [18,  1]  =  4.03703
  [19,  1]  =  -16.1481
  [20,  1]  =  6.05554
  [1 ,  2]  =  -12.1111
  [2 ,  2]  =  64.5925
  [3 ,  2]  =  -12.1111
  [12,  2]  =  40.3703
  ⋮
  [85, 95]  =  40.8162
  [94, 95]  =  -12.2449
  [95, 95]  =  65.3059
  [96, 95]  =  -12.2449
  [77, 96]  =  6.12243
  [78, 96]  =  -16.3265
  [79, 96]  =  4.08162
  [84, 96]  =  -16.3265
  [85, 96]  =  -12.2449
  [94, 96]  =  4.08162
  [95, 96]  =  -12.2449
  [96, 96]  =  12.2449

In [9]:
spy(A)

                   Sparsity Pattern
      ┌──────────────────────────────────────────┐    
    1 │⠿⣧⡄⠀⠀⢿⡄⠸⢿⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
      │⠀⠉⠿⣧⣀⠈⣧⡀⠈⠹⣧⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
      │⣤⣄⡀⠘⠛⣤⡘⢣⣤⣀⠀⠛⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⣀⡉⠉⠻⠶⣈⠻⢆⣈⠉⠛⠷⢆⠀⠀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠛⣷⣆⡀⠀⢻⡆⠘⢻⣶⡀⠀⢸⣆⠀⠛⣷⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠉⢿⣤⠀⢿⡄⠀⠈⠿⣧⡄⠹⣧⠀⠈⠹⢿⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠈⠉⠀⠈⠑⠲⢶⣄⡉⠱⣦⡉⠒⢶⣤⣈⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⢠⣤⠀⠉⠛⢣⠈⠛⣤⡄⠈⠙⢣⡄⠀⢠⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠈⠙⢻⣆⡀⠘⣷⡀⠉⢻⣶⣀⠈⢻⣆⠈⠛⣷⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠷⠆⠘⠷⣀⡀⠘⠻⢆⡀⠻⢆⡀⠀⠻⠶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⢶⣤⡈⠻⣦⡈⠛⠷⣦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠶⣦⠀⠈⠱⣦⠈⠱⣦⡄⠈⠉⢶⡄⠰⢶⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠹⢿⣤⡀⠹⣧⡀⠉⠿⣧⣀⠈⢿⡄⠈⠹⣧⣄⡀⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠃⠀⠘⢣⣄⡀⠘⠛⣤⡀⢣⣤⣀⠀⠛⠃⠀⠀⠀⠀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡉⠛⠷⠤⣈⠻⢆⣈⠙⠷⠦⢄⡀⠀⣀⡀⠀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⣷⣆⡀⠀⢻⣆⠘⢻⣶⡀⠀⠘⣷⠀⠛⣷⣀⠀⠀│    
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

## Numerical examples

The [`IterativeSolvers.jl`](https://github.com/JuliaMath/IterativeSolvers.jl) package implements most commonly used iterative solvers.

### Generate test matrix

In [10]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random

Random.seed!(280)

n = 100
# Poisson matrix of dimension n^2=10000, pd and sparse
A = matrixdepot("poisson", n)
@show typeof(A)
# dense matrix representation of A
Afull = convert(Matrix, A)
@show typeof(Afull)
# sparsity level
count(!iszero, A) / length(A)

typeof(A) = SparseArrays.SparseMatrixCSC{Float64,Int64}
typeof(Afull) = Array{Float64,2}


0.000496

In [11]:
spy(A)

                      Sparsity Pattern
         ┌──────────────────────────────────────────┐    
       1 │⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
         │⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
         │⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

In [12]:
# storage difference
Base.summarysize(A), Base.summarysize(Afull)

(873768, 800000040)

## Matrix-vector muliplication

In [13]:
# randomly generated vector of length n^2
b = randn(n^2)
# dense matrix-vector multiplication
@benchmark $Afull * $b

BenchmarkTools.Trial: 
  memory estimate:  78.20 KiB
  allocs estimate:  2
  --------------
  minimum time:     37.643 ms (0.00% GC)
  median time:      47.193 ms (0.00% GC)
  mean time:        47.269 ms (0.00% GC)
  maximum time:     50.683 ms (0.00% GC)
  --------------
  samples:          106
  evals/sample:     1

In [14]:
# sparse matrix-vector multiplication
@benchmark $A * $b

BenchmarkTools.Trial: 
  memory estimate:  78.20 KiB
  allocs estimate:  2
  --------------
  minimum time:     55.485 μs (0.00% GC)
  median time:      88.148 μs (0.00% GC)
  mean time:        100.386 μs (9.31% GC)
  maximum time:     3.186 ms (96.26% GC)
  --------------
  samples:          10000
  evals/sample:     1

### Dense solve via Cholesky

In [15]:
# record the Cholesky solution
xchol = cholesky(Afull) \ b;

In [16]:
# dense solve via Cholesky
@benchmark cholesky($Afull) \ $b

BenchmarkTools.Trial: 
  memory estimate:  763.02 MiB
  allocs estimate:  8
  --------------
  minimum time:     3.359 s (0.33% GC)
  median time:      3.450 s (1.59% GC)
  mean time:        3.450 s (1.59% GC)
  maximum time:     3.540 s (2.79% GC)
  --------------
  samples:          2
  evals/sample:     1

### Jacobi solver

It seems that Jacobi solver doesn't give the correct answer.

In [17]:
xjacobi = jacobi(A, b)
@show norm(xjacobi - xchol)

norm(xjacobi - xchol) = 533.315106586665


533.315106586665

Reading documentation we found that the default value of `maxiter` is 10. A couple trial runs shows that 30000 Jacobi iterations are enough to get an accurate solution.

In [18]:
xjacobi = jacobi(A, b, maxiter=30000)
@show norm(xjacobi - xchol)

norm(xjacobi - xchol) = 0.00012526024863889176


0.00012526024863889176

In [19]:
@benchmark jacobi($A, $b, maxiter=30000)

BenchmarkTools.Trial: 
  memory estimate:  234.72 KiB
  allocs estimate:  9
  --------------
  minimum time:     2.168 s (0.00% GC)
  median time:      2.200 s (0.00% GC)
  mean time:        2.193 s (0.00% GC)
  maximum time:     2.211 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

## Gauss-Seidal

In [20]:
# Gauss-Seidel solution is fairly close to Cholesky solution
xgs = gauss_seidel(A, b, maxiter=15000)
@show norm(xgs - xchol)

norm(xgs - xchol) = 0.0001245846635425542


0.0001245846635425542

In [21]:
@benchmark gauss_seidel($A, $b, maxiter=15000)

BenchmarkTools.Trial: 
  memory estimate:  156.55 KiB
  allocs estimate:  8
  --------------
  minimum time:     1.709 s (0.00% GC)
  median time:      1.791 s (0.00% GC)
  mean time:        1.768 s (0.00% GC)
  maximum time:     1.803 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

### SOR

In [22]:
# symmetric SOR with ω=0.75
xsor = ssor(A, b, 0.75, maxiter=10000)
@show norm(xsor - xchol)

norm(xsor - xchol) = 0.0022945523203956853


0.0022945523203956853

In [23]:
@benchmark sor($A, $b, 0.75, maxiter=15000)

BenchmarkTools.Trial: 
  memory estimate:  703.52 KiB
  allocs estimate:  30010
  --------------
  minimum time:     2.013 s (0.00% GC)
  median time:      2.080 s (0.00% GC)
  mean time:        2.074 s (0.00% GC)
  maximum time:     2.131 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

### Conjugate Gradient

In [24]:
# conjugate gradient
xcg = cg(A, b)
@show norm(xcg - xchol)

norm(xcg - xchol) = 1.1005386920768871e-5


1.1005386920768871e-5

In [25]:
@benchmark cg($A, $b)

BenchmarkTools.Trial: 
  memory estimate:  391.89 KiB
  allocs estimate:  23
  --------------
  minimum time:     18.031 ms (0.00% GC)
  median time:      19.166 ms (0.00% GC)
  mean time:        19.473 ms (0.20% GC)
  maximum time:     31.813 ms (0.00% GC)
  --------------
  samples:          257
  evals/sample:     1